In [13]:
import pandas as pd
from pathlib import Path

In [14]:
color_map = pd.read_csv("../data/color_map.csv")
color_map = color_map.melt(id_vars="entity", var_name="level", value_name="Color")

In [15]:
rank_to_level = {
    1: "highest",
    2: "high_medium",
    3: "low_medium",
    4: "lowest",
}

In [16]:
def get_category_mapping(categories):
    """Returns a dictionary mapping category names to the generic names in the color map."""
    generic_names = ["Cat1", "Cat2", "Cat3", "Cat4"]
    return {
        specific: generic
        for specific, generic in zip(sorted(categories.unique()), generic_names)
    }

In [17]:
all_df = []
for file in Path("out").glob("*.csv"):
    df = pd.read_csv(file).drop(columns=["Number of i-cords", "Color"], errors="ignore")
    df["level"] = df["Rank"].map(rank_to_level)
    df["Number of i-cords"] = df["Quartile"]
    if file.name.startswith("borrowed"):
        left_key = "From"
    else:
        df["Generic category"] = df["From"].map(get_category_mapping(df["From"]))
        left_key = "Generic category"
    df = df.merge(
        right=color_map,
        left_on=[left_key, "level"],
        right_on=["entity", "level"],
    ).drop(columns=["entity", "level"])
    df["Color"] = df["Color"].str.lower()
    df.to_csv(file, index=None)
    all_df.append(df)

In [18]:
combined = pd.concat(all_df)
combined.to_csv("out/combined.csv", index=None)